In [5]:
from PIL import Image
from utils.utils import AverageMeter
from utils.metrics import ssim, fid, inception_score
import os
import torch
from torchvision import transforms
import time

In [6]:
input_dir = '/opt/ml/data/dress_code/lower_body/images'         # 013563_0.jpg
output_dir = '/opt/ml/final-project-level3-cv-12/DAFlow/results/lower_test_pair/vis_viton_out'     # 050192_0.jpg

In [29]:
totensor = transforms.ToTensor()
resize = transforms.Resize((512, 384), interpolation=2)
#inputs = []       # FID
outputs = []
metric_ssim = AverageMeter()
metric_fid = AverageMeter()

# get list
names = os.listdir(output_dir)
N = len(names)

# ssim, fid
tall0 = time.time()
for i, name in enumerate(names):
    tall = time.time()
    if i%300==0 and i!=0: print(f"[{i}/{N}] {tall-tall0:.1f}s -> {(((tall-tall0)/i)*N):.1f}s")
    input = resize(totensor(Image.open(os.path.join(input_dir, name)))).cuda()
    output = totensor(Image.open(os.path.join(output_dir, name))).cuda()
    #inputs.append(input)   # FID
    outputs.append(output)
    
    metric_ssim.update(ssim(input.unsqueeze(0), output.unsqueeze(0)).item(), n=1)

# inputs  = torch.stack(inputs, 0)   # FID
outputs = torch.stack(outputs, 0)

# metric_fid = fid(inputs, outputs, batch_size=4)   # FID

metric_is = inception_score(outputs)

print(f"\nSSIM\t{metric_ssim.avg:.4f}")
#print(f"FID\t{metric_fid:.4f}")
print(f"IS\t{metric_is[0]:.4f}")


[300/1800] 5.3s -> 31.7s
[600/1800] 10.3s -> 30.9s
[900/1800] 15.8s -> 31.7s
[1200/1800] 20.8s -> 31.2s
[1500/1800] 25.2s -> 30.3s

SSIM	0.7803
IS	1.3001
